order_date를 토대로 시계열 데이터를 확보하여 수요 예측의 feature로 사용하려 함

이 코드를 실행하면 df의 컬럼에는 아래와 같이 생성됨
- baseline : lag_1, lag_7, rolling_7
- 요일 Feature : day_of_week_1 ~ day_of_week_6
- 월 Feature : month_2 ~ month_12

🎯 목적
- “요일이 진짜 영향 있나?”
- “월 효과가 수요에 얼마나 작용하나?”  
-> 설명 가능한 근거 확보

In [8]:
import pandas as pd

# ---------- 데이터셋 불러오기 ----------
df = pd.read_csv("../../data/blinkit-dataset/blinkit_master_data_eda_mk_251224.csv")


# ---------- order_date 타입 변경 ----------
df['order_date'] = pd.to_datetime(df['order_date'])


# ---------- 요일 feature ----------
df['day_of_week'] = df['order_date'].dt.day_of_week # 0=월요일, 6=일요일


# ---------- one-hot encoding ----------
df = pd.get_dummies(
    df,
    columns=['day_of_week'],
    drop_first=True
)


# ---------- 월 feature ----------
df['month'] = df['order_date'].dt.month

df = pd.get_dummies(
    df,
    columns=['month'],
    drop_first=True
)


# ---------- 컬럼 정의 함수 ----------
def add_time_features(df):
    df = df.copy()

    df['order_date'] = pd.to_datetime(df['order_date'])

    df['day_of_week'] = df['order_date'].dt.day_of_week
    df['month'] = df['order_date'].dt.month

    df = pd.get_dummies(
        df,
        columns=['day_of_week', 'month'],
        drop_first=True
    )

    return df

df.columns

Index(['order_id', 'customer_id', 'order_date', 'promised_delivery_time',
       'actual_delivery_time', 'delivery_status', 'order_total',
       'payment_method', 'delivery_partner_id', 'store_id', 'product_id',
       'quantity', 'unit_price', 'product_name', 'category', 'brand', 'price',
       'mrp', 'margin_percentage', 'shelf_life_days', 'min_stock_level',
       'max_stock_level', 'delivery_time_minutes', 'distance_km',
       'reasons_if_delayed', 'feedback_id', 'rating', 'feedback_text',
       'feedback_category', 'sentiment', 'feedback_date', 'customer_name',
       'email', 'phone', 'address', 'area', 'pincode', 'registration_date',
       'customer_segment', 'total_orders', 'avg_order_value', 'day_of_week_1',
       'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'day_of_week_5',
       'day_of_week_6', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [9]:
df

,order_id,customer_id,order_date,promised_delivery_time,actual_delivery_time,delivery_status,order_total,payment_method,delivery_partner_id,store_id,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1961864118,30065862,2024-07-17,2024-07-17 08:52:01,2024-07-17 08:47:01,On Time,3197.07,Cash,63230,4771,...,False,False,False,False,True,False,False,False,False,False
1,1549769649,9573071,2024-05-28,2024-05-28 13:25:29,2024-05-28 13:27:29,On Time,976.55,Cash,14983,7534,...,False,False,True,False,False,False,False,False,False,False
2,9185164487,45477575,2024-09-23,2024-09-23 13:25:12,2024-09-23 13:29:12,On Time,839.05,UPI,39859,9886,...,False,False,False,False,False,False,True,False,False,False
3,9644738826,88067569,2023-11-24,2023-11-24 16:34:56,2023-11-24 16:33:56,On Time,440.23,Card,61497,7917,...,False,False,False,False,False,False,False,False,True,False
4,5427684290,83298567,2023-11-20,2023-11-20 05:17:39,2023-11-20 05:18:39,On Time,2526.68,Cash,84315,2741,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1669690997,62600289,2023-12-25,2023-12-25 16:05:20,2023-12-25 16:10:20,On Time,1132.33,Cash,90914,1587,...,False,False,False,False,False,False,False,False,False,True
4996,8340761903,53640286,2023-11-27,2023-11-27 09:38:43,2023-11-27 09:36:43,On Time,2372.01,Cash,27952,3458,...,False,False,False,False,False,False,False,False,True,False
4997,5936301790,87059497,2024-06-21,2024-06-21 19:23:09,2024-06-21 19:26:09,On Time,3158.35,Cash,9590,7424,...,False,False,False,True,False,False,False,False,False,False
4998,5710579377,67310893,2024-06-06,2024-06-06 15:12:13,2024-06-06 15:10:13,On Time,1918.92,UPI,29940,6128,...,False,False,False,True,False,False,False,False,False,False


In [10]:
df.to_csv(
    "../../data/blinkit-dataset/blinkit_master_data_daily_feature_mk_251226.csv",
    index=False, # 인덱스 컬럼 안 생기게
    encoding="utf-8-sig" # 엑셀에서도 잘 열리게
    )